In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
from matplotlib import pyplot as plt

In [102]:
"""TOP 10 COUNTRIES WITH MOST PROJECTS"""

with open('data/world_bank_projects.json') as file:
    data = pd.read_json(file)

    #group entries together by their respective countries
    data = data.set_index(['countryname'])
    data = data.sort_index()
    
    #create list of unique countries from index: countries
    countries = set(data.index)
    #create a dictionary of the countries to their numbers of projects
    country_proj_counts = dict.fromkeys(countries)
        
    for country in countries:
        country_proj_counts[country] = sum(data.index == country)
        
    #convert country_proj_counts from dict to dataframe    
    country_proj_counts = pd.DataFrame.from_dict(country_proj_counts, orient='index')
    
    #sort dict by number of projects
    country_proj_counts = country_proj_counts.sort_values(0, ascending=False)
    top10 = country_proj_counts.head(10).to_string(header=False)
    top10_plot = country_proj_counts.head(10)
    top10_plot.columns = ['Counts']
    #filter out top 10 countries and print them one by one, with ascending ranking on the side
    print('Top 10 Countries with Most Projects: ')
    print(top10_plot)
    #plt.hist(top10_plot, normed=True, bins=10)
    '''Further improvement: create visuals(histogram)'''

Top 10 Countries with Most Projects: 
                                 Counts
Republic of Indonesia                19
People's Republic of China           19
Socialist Republic of Vietnam        17
Republic of India                    16
Republic of Yemen                    13
Nepal                                12
Kingdom of Morocco                   12
People's Republic of Bangladesh      12
Republic of Mozambique               11
Africa                               11
<class 'pandas.core.frame.DataFrame'>


In [9]:
"""TOP 10 MOST COMMON PROJECTS"""

file = 'data/world_bank_projects.json'
data = pd.read_json(file)
    
#index projects by their ids
data = data.set_index(['id']).sort_index()
#data = data.sort_index()

#create a big list of all dictionaries inside data['mjtheme_namecode']
themes = data['mjtheme_namecode']

#delete duplicates inside the dictionaries at themes: fixed_themes
fixed_themes = []
for theme in themes:
    fixed = [dict(t) for t in {tuple(d.items()) for d in theme}]
    fixed_themes.append(fixed)


#create a big list of all the themes of each project: all_themes
all_themes = []
for d in fixed_themes:
    [all_themes.append(a) for a in d]
    
#create two lists, one for 'code' column and one for 'name' column
codes = []
names = []
for i, dictionary in enumerate(all_themes):
    codes.append(all_themes[i]['code'])
    names.append(all_themes[i]['name'])
#replace all empty string values with None
names = [None if name == '' else name for name in names]


#populate new dataframe with the two lists: table
theme_table = pd.DataFrame(names)
code_table = pd.DataFrame(codes)
table = pd.concat([theme_table, code_table], axis=1)
table.columns = ['Names', 'Codes']


#create a reference table and dictionary of codes and their respective names, no duplicates
ref_table = table.sort_values('Codes', na_position='last').drop_duplicates(subset='Names').set_index('Codes')
ref_table = ref_table[pd.notnull(ref_table['Names'])]
ref_table.index = ref_table.index.astype(int)
ref_table = ref_table.sort_index()
ref_dict = ref_table.to_dict()
ref_dict = ref_dict['Names']

#replace Nones in Names column with their respective code names in the ref_table
for index, theme in table.iterrows():
    if theme['Names'] == None:
        theme['Names'] = ref_dict[int(theme['Codes'])]


#count number of occurrences for each theme and code
name_counts = table.groupby('Names').size().sort_values(ascending=False)
#name_counts = name_counts.sort_values(ascending=False)
code_counts = table.groupby('Codes').size().sort_values(ascending=False)
#code_counts = code_counts.sort_values(ascending=False)
'''problem: some names are not populated, go fix the tables (now replaced with Nones)'''

print('TOP 10 PROJECT THEMES:')
print(name_counts.head(10))


TOP 10 PROJECT THEMES:
Names
Environment and natural resources management    162
Rural development                               149
Public sector governance                        141
Human development                               131
Social protection and risk management           120
Financial and private sector development        120
Social dev/gender/inclusion                     114
Trade and integration                            61
Urban development                                40
Economic management                              33
dtype: int64
